In [1]:
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.config("spark.driver.memory", "20g").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/03 13:48:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
from pyspark.ml.image import ImageSchema

# Load images
data_home = "../../photos/Flicker8k_Dataset"
img_df = spark.read.format("image").load(data_home)

In [13]:
img_df

DataFrame[image: struct<origin:string,height:int,width:int,nChannels:int,mode:int,data:binary>]

In [17]:
# let's check one sample
img_df.show(1, truncate=False)

IOPub data rate exceeded.                                                       
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [8]:
import sys
sys.path.append("../src")

In [10]:
from key import hf_key
from utils import ImageTopicExtractor, set_topic_explainer_pipe, set_image_sumarizer_pipe

In [15]:
from huggingface_hub import login
from torch import bfloat16

In [16]:
login(hf_key)

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/gv2167/.cache/huggingface/token
Login successful


In [18]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline

# def image_summarizer(df):
topic_explainer_pipeline = utils.set_topic_explainer_pipe()
login(hf_key)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)
model_id = "meta-llama/LLama-2-7b-chat-hf"

description_tokenizer = AutoTokenizer.from_pretrained(model_id)
description_model = AutoModelFromCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto"
)

description_generator = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/gv2167/.cache/huggingface/token
Login successful


NameError: name 'bfloat16' is not defined

In [ ]:
topic_extractor 
title_abs = [d.title + tokenizer.sep_token + d.abstract  for idx, d in df.iterrows()]

def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
      yield lst[i:i + n]

batch_size = 20
embeddings_chunks = []
for c in chunks(title_abs, batch_size):
    # preprocess the input
    inputs = tokenizer(c, padding=True, truncation=True, return_tensors="pt", max_length=512)
    result = model(**inputs)
    # take the first token in the batch as the embedding
    embeddings = result.last_hidden_state[:, 0, :].cpu().detach().numpy()
    embeddings_chunks.append(embeddings)

embeddings = np.concatenate(embeddings_chunks)

return_df = (
    df[["id"]]
    .assign(embedding=list(embeddings))
)
# return return_df

In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType, FloatType
from pyspark.sql.functions import spark_partition_id

embed_schema = StructType(
    [
        StructField("id", StringType(), True),
        StructField("embedding", ArrayType(FloatType()), True)
    ]
)

embed_df = (
    text_to_embed_df
    .groupBy(spark_partition_id().alias("_pid"))
    .applyInPandas(embed_func, embed_schema)
)

embed_df.show()

In [ ]:
# Preprocess input image
df = df.select("image.origin", preprocess_input("image.data").alias("features"))

# Extract image features
df = model.transform(df)